In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn

## 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier  #### random forest algorithm 사용.

from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [3]:
## 데이터 탐색. 

In [4]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
df
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


(891, 12)

In [6]:
df.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:

## 평균값
d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)

## 최빈값
d_mode = df["Embarked"].mode()[0]
df["Embarked"].fillna(d_mode, inplace=True)

## 파생 변수 생성
df["FamilySize"] = df["SibSp"] + df["Parch"]

C:\Users\cst_3\AppData\Local\Temp\ipykernel_9404\3244494457.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(d_mean, inplace=True)
C:\Users\cst_3\AppData\Local\Temp\ipykernel_9404\3244494457.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when d

In [8]:
## one hot encoding  vs.  label encoding 
## 지금은 one hot encoding 방식을 사용해 보자.  Label encoding은 잘 사용하지 않지만 tree 구조에서는 사용하는 경우가 있다.

In [9]:
#sex 컬럼의 값을 1과 0 으로 원-핫 인코딩 한다.
onehot_sex = pd.get_dummies(df["Sex"])   ## 가 변수를 생성
df = pd.concat([df, onehot_sex], axis = 1) ## object value 이름으로 새로운 field 생성

#Embarked 컬럼의 값을 원-핫 인코딩 한다.
onehot_embarked = pd.get_dummies(df["Embarked"])
df = pd.concat([df, onehot_embarked], axis=1)  ## object value 이름으로 새로운 field 생성



In [10]:
onehot_sex

,female,male
0,False,True
1,True,False
2,True,False
3,True,False
4,False,True
...,...,...
886,False,True
887,True,False
888,True,False
889,False,True


In [11]:
onehot_embarked

,C,Q,S
0,False,False,True
1,True,False,False
2,False,False,True
3,False,False,True
4,False,False,True
...,...,...,...
886,False,False,True
887,False,False,True
888,False,False,True
889,True,False,False


In [ ]:
#X = df[["Pclass", "Age", "Fare", "FamilySize","female", "male", "C", "Q", "S"]]   # cabin, ticket을 뺀다.  text는 영향성이 없다고 사전 판단.
#y = df ["Survived"]

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=10)
##print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)

In [ ]:
#SVM 객체 생성 
#sv = svm.SVC(kernel='rbf')  # kernel의 종류:linear, poly rbf ... 
# sv = svm.SVC(kernel='rbf', C=1, gamma=0.1)  # C 비용 , gamma : 허용 ㅍ준 편차
#sv.fit(X_train, y_train)

In [ ]:
#pred = sv.predict(X_test)  ## 예측 수행

In [ ]:
#pred = sv.predict(X_test)

In [ ]:
## 모델 성능 - 정확도 측정

In [ ]:
#from sklearn.metrics import accuracy_score
#acc = accuracy_score(y_test, pred)
#print(acc)

In [ ]:
## 모델 성능 평가 - Confusion Matrix 계산

In [ ]:
#from sklearn.metrics import confusion_matrix
#mat = confusion_matrix(y_test, pred)   ##실제 분류가 어떻게 되었을 까?
#print(mat)

In [ ]:
## 평가 지표 계산
##from sklearn.metrics import classification_report
#rpt = classification_report(y_test, pred)
#print(rpt)

# 

In [ ]:
#분석 data set 준비
# X는 독립변수(설명변수), y는 종속 변수(목표 변수)
X = df[["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize"]]   # cabin, ticket을 뺀다.  text는 영향성이 없다고 사전 판단.
y = df ["Survived"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=10)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [36]:
#RandomForestClasssifier 객체 생성
rf = RandomForestClassifier(n_estimators = 50, max_depth=3, random_state=20)   ## RandomForest 입력 변수 값  조사 .. 
rf.fit(X_train, y_train)

ValueError: could not convert string to float: 'male'

In [34]:
pred = rf.predict(X_test)

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'